# Tâche #1 : Classification d'incidents avec un réseau *feedforward* et des *embeddings* Spacy

On reprend la classification des descriptions d’accidents du premier travail. Le corpus de textes contient 3 partitions : 
-	un fichier d’entraînement -  data/incidents_train.json
-	un fichier de validation -  data/incidents_dev.json
-	un fichier de test - data/incidents_test.json

Entraînez un modèle de réseau de neurones de type feedforward multicouche (MLP) avec plongements de mots pour déterminer le type d’un incident à partir de sa description. 

Voici les consignes pour cette tâche : 

-	Nom du notebook : mlp.ipynb
-	Tokenisation : Utilisation de Spacy. 
-	Plongements de mots : Ceux de Spacy. 
-	Normalisation : Aucune normalisation. 
-	Agrégation des plongements de mots : Comparer les approches max, average et min pooling. 
-	Structure du réseau : 1 seule couche cachée dont vous choisirez la taille (à expliquer). 
-	Présentez clairement vos résultats et faites-en l’analyse. En cas de doute, inspirez-vous de ce qui a été fait dans le travail pratique #1. 

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [7]:
import json
import spacy
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



In [6]:
# Charger le modèle de langue de spaCy 
# !python -m spacy download fr_core_news_md pour télécharger
nlp = spacy.load('fr_core_news_md')

# Définir le Dataset
class IncidentDataset(Dataset):
    def __init__(self, file):
        with open(file, 'r') as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = int(self.data[idx]['label'])
        # Tokenisation et plongements de mots avec spaCy
        doc = nlp(text)
        embeddings = [token.vector for token in doc]
        return torch.tensor(embeddings), label


In [11]:
# print current directory
import os
print(os.getcwd())

c:\Users\taha\Coding\Academic-Projects\Artificial Neural Networks\Laval


In [10]:

# Charger les données
train_data = IncidentDataset(r'Academic-Projects\Artificial Neural Networks\Laval\data\incidents_train.json')
dev_data = IncidentDataset(r'Academic-Projects\Artificial Neural Networks\Laval\data\incidents_dev.json')
test_data = IncidentDataset(r'Academic-Projects\Artificial Neural Networks\Laval\data\incidents_test.json')

# Définir le DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)



FileNotFoundError: [Errno 2] No such file or directory: '.\\Academic-Projects\\Artificial Neural Networks\\Laval\\data\\incidents_train.json'

In [ ]:
# Définir le MLP
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.max(x, dim=0)[0]  # max pooling
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialiser le MLP
model = MLP(300, 100, 6)  # 300 = dimension des plongements de mots de spaCy, 100 = taille de la couche cachée, 6 = nombre de classes

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Boucle d'entraînement
for epoch in range(10):  # 10 = nombre d'époques
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

## 2. Gestion de plongements de mots (*embeddings*)

## 3. Création de modèle(s)

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

## 5. Entraînement de modèle(s)

## 6. Évaluation et analyse de résultats